In [103]:
import pickle
import pandas as pd
import csv

results = list()

datasetFile = open("fnmr/dataset/fnmr_dataset_lfw.pickle", 'rb')
dataset = pickle.load(datasetFile)

resultFile = open("fnmr/results/openface_fnmr_lfw.pickle", 'rb')
results.append(pickle.load(resultFile))

resultFile = open("fnmr/results/arcface_fnmr_lfw.pickle", 'rb')
results.append(pickle.load(resultFile))

resultFile = open("fnmr/results/facenet_fnmr_lfw.pickle", 'rb')
results.append(pickle.load(resultFile))

resultFile = open("fnmr/results/vggface_fnmr_lfw.pickle", 'rb')
results.append(pickle.load(resultFile))

resultFile = open("fnmr/results/faceapi_fnmr_lfw.pickle", 'rb')
faceapiDataset = (pickle.load(resultFile))


resultDict = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict()
}

for result in results:
    for algorithm in result.keys():
        if result[algorithm]:
            resultDict[algorithm] = result[algorithm]

In [104]:
labels = pd.read_csv(r'./labels_lfw.csv')
names = list()
labelDict = dict()

with open('names_lfw.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)[0]
    for name in data:
        names.append(name.split('\\')[0])


for index, row in labels.iterrows():
    if names[index] in labelDict.keys():
        cont = False
        for label in labelDict[names[index]].keys():
            if labelDict[names[index]][label] == True:
                cont = True
        if cont:
            continue
    labelDict[names[index]] = dict()
    labelDict[names[index]]["Asian"] = bool(row["Asian"])
    labelDict[names[index]]["White"] = bool(row["White"])
    labelDict[names[index]]["Black"] = bool(row["Black"])
    labelDict[names[index]]["Indian"] = bool(row["Indian"])
    

In [105]:
processDict = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict()
}

for algorithm in resultDict.keys():
    for index in range(len(dataset)):
        referenceSubject = (dataset[index][0].split('/'))[0]
        comparisonSubject = (dataset[index][1].split('/'))[0]

        pairNumber = 'pair_' + str(index + 1)
        if not(referenceSubject in processDict[algorithm].keys()):
            processDict[algorithm][referenceSubject] = list()
        processDict[algorithm][referenceSubject].append(resultDict[algorithm][pairNumber])

faceAPIResult = dict()

for comparison in faceapiDataset:
    parsedComparison = comparison.split(" ")
    parsedComparison[0] = parsedComparison[0].replace("/home/ubuntu/lfw/", "")
    parsedComparison[1] = parsedComparison[1].replace("/home/ubuntu/lfw/", "")
    parsedComparison[2].replace("undefined", "1")
    try: 
        parsedComparison[2] = float(parsedComparison[2])
    except:
        parsedComparison[2] = float(1)

    
    referenceSubject = (parsedComparison[0].split('/'))[0]
    comparisonSubject = (parsedComparison[1].split('/'))[0]
    distance = parsedComparison[2]

    comparisonDict = dict()
    comparisonDict["distance"] = distance
    if distance >= float(.6):
        comparisonDict["verified"] = False
    else:
        comparisonDict["verified"] = True

    if not(referenceSubject in faceAPIResult.keys()):
        faceAPIResult[referenceSubject] = list()
    faceAPIResult[referenceSubject].append(comparisonDict)

processDict["FaceAPI"] = faceAPIResult

In [106]:
verifiedDict = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict(),
    "FaceAPI": dict()
}


for algorithm in processDict.keys():
    for subject in processDict[algorithm].keys():
        for comparison in processDict[algorithm][subject]:
            if not(subject in verifiedDict[algorithm].keys()):
                verifiedDict[algorithm][subject] = list()
            verified = False
            if comparison["distance"] < .6:
                verified = True
            verifiedDict[algorithm][subject].append(verified)



In [107]:
verifiedStats = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict(),
    "FaceAPI": dict()
}

for algorithm in verifiedDict.keys():
    for subject in verifiedDict[algorithm].keys():
        for comparison in verifiedDict[algorithm][subject]:
            if not(subject in verifiedStats[algorithm].keys()):
                verifiedStats[algorithm][subject] = dict()
                verifiedStats[algorithm][subject]["tpr"] = 0
                verifiedStats[algorithm][subject]["fnr"] = 0
            if comparison:
                verifiedStats[algorithm][subject]["tpr"] = verifiedStats[algorithm][subject]["tpr"] + 1
            else:
                verifiedStats[algorithm][subject]["fnr"] = verifiedStats[algorithm][subject]["fnr"] + 1

Overall FPR and TNR by algorithm:

In [108]:
summaryVerifiedStats = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict(),
    "FaceAPI": dict()
}

for algorithm in verifiedStats.keys():

    summaryVerifiedStats[algorithm]["tpr"] = 0
    summaryVerifiedStats[algorithm]["fnr"] = 0

    for subject in verifiedStats[algorithm].keys():
        summaryVerifiedStats[algorithm]["tpr"] = verifiedStats[algorithm][subject]["tpr"] + summaryVerifiedStats[algorithm]["tpr"]
        summaryVerifiedStats[algorithm]["fnr"] = verifiedStats[algorithm][subject]["fnr"] + summaryVerifiedStats[algorithm]["fnr"]

print("FNMR Results:")
for algorithm in verifiedStats.keys():
    print()
    print(algorithm)
    print("FN:  " + str(summaryVerifiedStats[algorithm]["fnr"]))
    print("TP:  " + str(summaryVerifiedStats[algorithm]["tpr"]))  
    print("FNMR:  " + str(round(float(summaryVerifiedStats[algorithm]["fnr"])/float(summaryVerifiedStats[algorithm]["fnr"] + summaryVerifiedStats[algorithm]["tpr"]) * 100, 3)) + "%")  


FNMR Results:

Facenet
FN:  1073
TP:  6411
FNMR:  14.337%

VGG-Face
FN:  207
TP:  7277
FNMR:  2.766%

OpenFace
FN:  911
TP:  6573
FNMR:  12.173%

ArcFace
FN:  1763
TP:  5721
FNMR:  23.557%

FaceAPI
FN:  1640
TP:  5844
FNMR:  21.913%


In [109]:
summaryVerifiedStatsLabelled = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict(),
    "FaceAPI": dict()
}


for algorithm in verifiedStats.keys():
    for label in labelDict[names[0]].keys():
        if not(label in summaryVerifiedStatsLabelled[algorithm].keys()):
            summaryVerifiedStatsLabelled[algorithm][label] = dict()
        summaryVerifiedStatsLabelled[algorithm][label]["tpr"] = 0
        summaryVerifiedStatsLabelled[algorithm][label]["fnr"] = 0

    for subject in verifiedStats[algorithm].keys():
        labelledRace = ""
        for label in labelDict[subject].keys():
            if labelDict[subject][label]:
                labelledRace = label
        if labelledRace == "":
            continue
        summaryVerifiedStatsLabelled[algorithm][labelledRace]["tpr"] = verifiedStats[algorithm][subject]["tpr"] + summaryVerifiedStatsLabelled[algorithm][labelledRace]["tpr"]
        summaryVerifiedStatsLabelled[algorithm][labelledRace]["fnr"] = verifiedStats[algorithm][subject]["fnr"] + summaryVerifiedStatsLabelled[algorithm][labelledRace]["fnr"]


for algorithm in summaryVerifiedStatsLabelled.keys():
    for label in summaryVerifiedStatsLabelled[algorithm]:
        summaryVerifiedStatsLabelled[algorithm][label]["fnmr"] = str(round((float(summaryVerifiedStatsLabelled[algorithm][label]["fnr"])/float(summaryVerifiedStatsLabelled[algorithm][label]["fnr"] + summaryVerifiedStatsLabelled[algorithm][label]["tpr"]))*100,3)) + "%"

print("FNMR Results:")
for algorithm in verifiedStats.keys():
    print()
    print(algorithm)
    print("FN:  " + "\tBlack: " + str(summaryVerifiedStatsLabelled[algorithm]["Black"]["fnr"])+ "\tWhite: " + str(summaryVerifiedStatsLabelled[algorithm]["White"]["fnr"]) + "\tAsian: " + str(summaryVerifiedStatsLabelled[algorithm]["Asian"]["fnr"]) + "\tIndian: " + str(summaryVerifiedStatsLabelled[algorithm]["Indian"]["fnr"]))
    print("TP:  " + "\tBlack: " + str(summaryVerifiedStatsLabelled[algorithm]["Black"]["tpr"])+ "\tWhite: " + str(summaryVerifiedStatsLabelled[algorithm]["White"]["tpr"]) + "\tAsian: " + str(summaryVerifiedStatsLabelled[algorithm]["Asian"]["tpr"]) + "\tIndian: " + str(summaryVerifiedStatsLabelled[algorithm]["Indian"]["tpr"]))  
    print("FNMR:  " + "\tBlack: " + str(summaryVerifiedStatsLabelled[algorithm]["Black"]["fnmr"])+ "\tWhite: " + str(summaryVerifiedStatsLabelled[algorithm]["White"]["fnmr"]) + "\tAsian: " + str(summaryVerifiedStatsLabelled[algorithm]["Asian"]["fnmr"]) + "\tIndian: " + str(summaryVerifiedStatsLabelled[algorithm]["Indian"]["fnmr"]))



FNMR Results:

Facenet
FN:  	Black: 67	White: 890	Asian: 79	Indian: 24
TP:  	Black: 373	White: 5091	Asian: 749	Indian: 148
FNMR:  	Black: 15.227%	White: 14.88%	Asian: 9.541%	Indian: 13.953%

VGG-Face
FN:  	Black: 6	White: 175	Asian: 22	Indian: 1
TP:  	Black: 434	White: 5806	Asian: 806	Indian: 171
FNMR:  	Black: 1.364%	White: 2.926%	Asian: 2.657%	Indian: 0.581%

OpenFace
FN:  	Black: 56	White: 731	Asian: 85	Indian: 31
TP:  	Black: 384	White: 5250	Asian: 743	Indian: 141
FNMR:  	Black: 12.727%	White: 12.222%	Asian: 10.266%	Indian: 18.023%

ArcFace
FN:  	Black: 132	White: 1440	Asian: 126	Indian: 47
TP:  	Black: 308	White: 4541	Asian: 702	Indian: 125
FNMR:  	Black: 30.0%	White: 24.076%	Asian: 15.217%	Indian: 27.326%

FaceAPI
FN:  	Black: 103	White: 1341	Asian: 149	Indian: 36
TP:  	Black: 337	White: 4640	Asian: 679	Indian: 136
FNMR:  	Black: 23.409%	White: 22.421%	Asian: 17.995%	Indian: 20.93%
